In [1]:
import pandas as pd
import itertools
from ortools.linear_solver import pywraplp as OR

In [2]:
ex = pd.read_csv('fws_example.csv', index_col=0)

In [3]:
def fws(preferences, capcity, penalty, integer = False):
    """A model for solving a first-year writing seminar assignment problem.
    
    Args:
        preferences (pd.DataFrame): Preferred classes for each student.
        capcity (int): Capacity of the classroom.
        penalty (float): Penalty for not assigning a student to one of their top 5 picks.
    """
    STUDENTS = list(preferences.index) + ['dummy']             # students
    
    CLASSES = []
    for c in preferences.columns:
        CLASSES = CLASSES + list(preferences[c].unique())
    CLASSES = list(set(CLASSES))                               # classes
    
    edge_costs = {}                                 
    for s in preferences.index:
        for c in preferences:
            edge_costs[(s, preferences.at[int(s),c])] = int(c)
    EDGES = list(edge_costs)                                   # edges
    
    # add dummy edges
    dummy_edges = list(itertools.product([0], CLASSES))
    EDGES.extend(dummy_edges)          
    
    c = edge_costs                                             # costs
    # add dummy edge costs
    for edge in dummy_edges:
        c.update({edge : penalty})    
    
    # define model
    m = OR.Solver('fws', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {}    
    for i,j in EDGES:
        if integer:
            x[i,j] = m.IntVar(0, m.infinity(), ('(%d, %s)' % (i,j)))
        else:
            x[i,j] = m.NumVar(0, m.infinity(), ('(%d, %s)' % (i,j)))
        
    # objective function
    m.Minimize(sum(c[i,j]*x[i,j] for i,j in EDGES))
       
    # subject to: each student (not including the dummy) is assigned at most one class
    for k in STUDENTS:
        if k != 'dummy':
            m.Add(sum(x[i,j] for i,j in EDGES if i==k) <= 1)
        
    # subject to: each class is full
    for k in CLASSES:
        m.Add(sum(x[i,j] for i,j in EDGES if j==k) == capcity)
    
    return m,x

In [4]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [5]:
m,x = fws(ex, 2, 3, integer=True)
solve(m)

Solution:
Objective value = 11.0
(1, A) : 1.0
(1, B) : 0.0
(2, D) : 1.0
(2, C) : 0.0
(3, A) : 0.0
(3, C) : 1.0
(4, B) : 1.0
(4, D) : 0.0
(5, C) : 1.0
(5, B) : 0.0
(6, A) : 1.0
(6, B) : 0.0
(7, B) : 1.0
(7, A) : 0.0
(0, A) : 0.0
(0, C) : 0.0
(0, B) : 0.0
(0, D) : 1.0
